In [1]:
from pyspark.sql import SparkSession

# Construção da sessão do Spark
spark = SparkSession.builder.appName("Exercício Slide") \
    .config("spark.executor.memory", "1gb") \
    .getOrCreate()
    #.master("local") \
    
sc = spark.sparkContext

In [ ]:
# Rodar em caso de erro
sc.stop()

### Passo 1: Leitura dos arquivos

In [2]:
# Realização da leitura dos arquivos, alencando em variáveis para agosto e julho
rdd_agosto = sc.textFile("access_log_Aug95")
rdd_julho = sc.textFile("access_log_Jul95")

### Passo 2: Retirada dos caracteres que atrapalham a leitura do arquivo

In [37]:
# TESTE de limpeza de strings
step = rdd_agosto.take(1)
step = step[0]
step = step.replace(" -","").replace("[","")
step = step.replace("]","").replace('"','')
step = step.replace("GET ", "GET").replace(" HTTP","/HTTP")
step = step.split()
#step = step[1][:12]
step

['in24.inetnebr.com',
 '01/Aug/1995:00:00:010400',
 'GET/shuttle/missions/sts-68/news/sts-68-mcc-05.txt/HTTP/1.0',
 '200',
 '1839']

In [3]:
# Através do método map(), foi criada uma função chamada ajustes que passa por cada linha do arquivo editando:
# 1: ' -'
# 2: Retira os colchetes do timestamp
# 3: Retira as aspas da requisição
# 4: Alinha o início e o final da requisição retirando os espaços
# Como toda requisição começa com o termo GET e termina com HTTP, aproveitamos disso

def ajustes(lista):
    lista = lista.replace(" -","").replace("[","") \
                 .replace("]","").replace('"','') \
                 .replace("GET ", "GET").replace(" HTTP","/HTTP") \
                 .split()
    return lista

rdd_agosto_clean = rdd_agosto.map(lambda line: ajustes(line))
rdd_julho_clean = rdd_julho.map(lambda line: ajustes(line))

In [4]:
rdd_agosto_clean.take(2)

[['in24.inetnebr.com',
  '01/Aug/1995:00:00:010400',
  'GET/shuttle/missions/sts-68/news/sts-68-mcc-05.txt/HTTP/1.0',
  '200',
  '1839'],
 ['uplherc.upl.com', '01/Aug/1995:00:00:070400', 'GET//HTTP/1.0', '304', '0']]

In [5]:
rdd_julho_clean.take(2)

[['199.72.81.55',
  '01/Jul/1995:00:00:010400',
  'GET/history/apollo//HTTP/1.0',
  '200',
  '6245'],
 ['unicomp6.unicomp.net',
  '01/Jul/1995:00:00:060400',
  'GET/shuttle/countdown//HTTP/1.0',
  '200',
  '3985']]

### Passo 3: Com os arquivos limpos, é hora de filtrar e selecionar as opções

##### Colunas: Host, TimeStamp, Requisição, Codigo de retorno, Bytes

#### 1: Número de hosts únicos.
#### (Contar todos os valores únicos da coluna HOST do arquivo em agosto)

In [7]:
sorted(rdd_agosto_clean.map(lambda line: line[0]).countByValue().items())

[('***.novo.dk', 16),
 ('001.msy4.communique.net', 8),
 ('007.thegap.com', 4),
 ('01-dynamic-c.wokingham.luna.net', 31),
 ('01.ts01.zircon.net.au', 3),
 ('02-17-05.comsvc.calpoly.edu', 8),
 ('023.msy4.communique.net', 2),
 ('03-dynamic-c.wokingham.luna.net', 39),
 ('0321jona.jon.rpslmc.edu', 32),
 ('033.msy4.communique.net', 2),
 ('04-dynamic-c.wokingham.luna.net', 6),
 ('049.215.goodnet.com', 8),
 ('05-dynamic-c.wokingham.luna.net', 25),
 ('052.215.goodnet.com', 42),
 ('07mb369b.uni-duisburg.de', 6),
 ('08-dynamic-c.rotterdam.luna.net', 1),
 ('0875pr3e.pro.rpslmc.edu', 16),
 ('1.ts1.mnet.medstroms.se', 32),
 ('1.ts2.mnet.medstroms.se', 6),
 ('10.ts2.mnet.medstroms.se', 15),
 ('101.irri.cgiar.org', 17),
 ('1032015.ksc.nasa.gov', 186),
 ('1032by.sscl.uwo.ca', 5),
 ('107.tcom.co.uk', 6),
 ('10forward.ppp.america.com', 18),
 ('10md423.uni-duisburg.de', 39),
 ('11.ts1.mnet.medstroms.se', 12),
 ('110.the-spa.com', 4),
 ('118.32.med.umich.edu', 15),
 ('11client9.ihi.ku.dk', 1),
 ('12-102eb.i

#### 1.1: Número de hosts únicos.
#### (Contar todos os valores únicos da coluna HOST do arquivo em julho)

In [8]:
sorted(rdd_julho_clean.map(lambda line: line[0]).countByValue().items())

[('***.novo.dk', 16),
 ('007.thegap.com', 45),
 ('01-dynamic-c.rotterdam.luna.net', 1),
 ('01-dynamic-c.wokingham.luna.net', 28),
 ('02-dynamic-c.wokingham.luna.net', 13),
 ('03-dynamic-c.wokingham.luna.net', 15),
 ('04-dynamic-c.rotterdam.luna.net', 22),
 ('04-dynamic-c.wokingham.luna.net', 28),
 ('05-dynamic-c.rotterdam.luna.net', 26),
 ('05-dynamic-c.wokingham.luna.net', 9),
 ('06-dynamic-c.rotterdam.luna.net', 22),
 ('07-dynamic-c.rotterdam.luna.net', 4),
 ('0772jela.jelke.rpslmc.edu', 3),
 ('08-dynamic-c.rotterdam.luna.net', 4),
 ('086.msy4.communique.net', 11),
 ('0885-12.etech.fh-hamburg.de', 7),
 ('10-dynamic-c.rotterdam.luna.net', 4),
 ('10.salc.wsu.edu', 6),
 ('10.ts2.mnet.medstroms.se', 5),
 ('100.144.med.umich.edu', 23),
 ('10044.geo.net', 37),
 ('103.135.med.umich.edu', 9),
 ('1032015.ksc.nasa.gov', 89),
 ('1032ap.sscl.uwo.ca', 5),
 ('105.4.med.umich.edu', 4),
 ('109-02.ic.uva.nl', 2),
 ('10forward.ppp.america.com', 24),
 ('10md423.uni-duisburg.de', 9),
 ('11.ts2.mnet.meds

#### 2: O total de erros 404.
#### (Contar somente os valores únicos iguais a 404 em agosto)

In [ ]:
# Primeiro método testado:

# Cria-se uma lista para armazenar os erros
# Criamos um loop que percorre o rdd pela função take, em seu parametro passamos a quantidade total de linha
# Dentro do for, armazenamos os valores que correspondem aos erros de requisição

#erro_lista = []

#for x in rdd_agosto_clean.take(rdd_agosto_clean.count()):
#    erro_lista.append(x[3])
#total = erro_lista.count("404")

In [9]:
# Método utilizando filter + count
# Filtra os valores 404 na posição 3 da lista e conta

erro_agosto = rdd_agosto_clean.filter(lambda line: line[3] == "404").count()
print("Quantidade de erros 404 no mês de Agosto:", erro_agosto)

Quantidade de erros 404 no mês de Agosto: 10005


#### 2.1: O total de erros 404.
#### (Contar somente os valores únicos iguais a 404 em julho)

In [10]:
def verifica(lista):
    if len(lista) == 4:
        if lista[3] == "404":
            return lista[3]

erro_julho = rdd_julho_clean.filter(lambda line: verifica(line)).count()
print("Quantidade de erros 404 no mês de Julho:", erro_julho)

Quantidade de erros 404 no mês de Julho: 10769


#### 3: Os 5 URLS que mais causaram erro 404 em agosto

In [ ]:
# Realiza um filtro das linhas que possuem erro 404
# Faz um mapeamento pegando somente o primeiro e o último valor
# Faz um mapeamento chave valor contando os pares de chave
# Realiza uma ordenação pela quantidade do maior para o menor

In [11]:
dict_url = rdd_agosto_clean.filter(lambda line: line[3] == "404") \
                           .map(lambda line: line[0::3]) \
                           .map(lambda line: (line[0],line[1])).countByValue()

In [12]:
sorted(dict_url.items(), key= lambda line: line[1], reverse=True)

[(('dialip-217.den.mmc.com', '404'), 62),
 (('piweba3y.prodigy.com', '404'), 47),
 (('155.148.25.4', '404'), 44),
 (('maz3.maz.net', '404'), 39),
 (('gate.barr.com', '404'), 38),
 (('ts8-1.westwood.ts.ucla.edu', '404'), 37),
 (('nexus.mlckew.edu.au', '404'), 37),
 (('204.62.245.32', '404'), 37),
 (('m38-370-9.mit.edu', '404'), 37),
 (('scooter.pa-x.dec.com', '404'), 35),
 (('reddragon.ksc.nasa.gov', '404'), 33),
 (('www-c4.proxy.aol.com', '404'), 32),
 (('piweba5y.prodigy.com', '404'), 31),
 (('www-d4.proxy.aol.com', '404'), 30),
 (('piweba4y.prodigy.com', '404'), 30),
 (('internet-gw.watson.ibm.com', '404'), 29),
 (('163.206.104.34', '404'), 28),
 (('unidata.com', '404'), 28),
 (('spica.sci.isas.ac.jp', '404'), 27),
 (('www-d2.proxy.aol.com', '404'), 26),
 (('203.13.168.17', '404'), 25),
 (('203.13.168.24', '404'), 25),
 (('www-d1.proxy.aol.com', '404'), 23),
 (('www-c2.proxy.aol.com', '404'), 23),
 (('crl5.crl.com', '404'), 23),
 (('piweba1y.prodigy.com', '404'), 22),
 (('onramp2-9.o

#### 3.1: Os 5 URLS que mais causaram erro 404 em julho

In [13]:
# Utiliza a função verifica para trazer os valores iguais a 404

dict_url = rdd_julho_clean.filter(lambda line: verifica(line)) \
                          .map(lambda line: line[0::3]) \
                          .map(lambda line: (line[0],line[1])).countByValue()

In [14]:
sorted(dict_url.items(), key= lambda line: line[1], reverse=True)

[(('hoohoo.ncsa.uiuc.edu', '404'), 251),
 (('jbiagioni.npt.nuwc.navy.mil', '404'), 131),
 (('piweba3y.prodigy.com', '404'), 110),
 (('piweba1y.prodigy.com', '404'), 92),
 (('phaelon.ksc.nasa.gov', '404'), 64),
 (('www-d4.proxy.aol.com', '404'), 61),
 (('piweba4y.prodigy.com', '404'), 56),
 (('monarch.eng.buffalo.edu', '404'), 56),
 (('alyssa.prodigy.com', '404'), 54),
 (('titan02f', '404'), 53),
 (('www-a2.proxy.aol.com', '404'), 52),
 (('www-b4.proxy.aol.com', '404'), 48),
 (('www-b6.proxy.aol.com', '404'), 44),
 (('www-b3.proxy.aol.com', '404'), 43),
 (('tearnest2.stpaul.ncr.com', '404'), 42),
 (('www-b2.proxy.aol.com', '404'), 41),
 (('www-d1.proxy.aol.com', '404'), 41),
 (('www-a1.proxy.aol.com', '404'), 38),
 (('piweba2y.prodigy.com', '404'), 38),
 (('www-d3.proxy.aol.com', '404'), 38),
 (('scooter.pa-x.dec.com', '404'), 34),
 (('proxy.austin.ibm.com', '404'), 34),
 (('www-d2.proxy.aol.com', '404'), 33),
 (('www-b5.proxy.aol.com', '404'), 32),
 (('monolith.uunet.ca', '404'), 31),


#### 4: Quantidade de erros 404 por dia em agosto
#### Recuperar os dias do campo timestamp e contar os erros 404

In [39]:
def ajusta_time(lista):
    lista[0] = lista[0][:11]
    return lista

erro_diaAG = rdd_agosto_clean.filter(lambda line: verifica(line)) \
                             .map(lambda line: line[1::2]) \
                             .map(lambda line: ajusta_time(line)) \
                             .map(lambda line: line[0]).countByValue()

In [41]:
erro_diaAG

defaultdict(int,
            {'01/Aug/1995': 243,
             '03/Aug/1995': 303,
             '04/Aug/1995': 346,
             '05/Aug/1995': 233,
             '06/Aug/1995': 372,
             '07/Aug/1995': 532,
             '08/Aug/1995': 378,
             '09/Aug/1995': 278,
             '10/Aug/1995': 311,
             '11/Aug/1995': 262,
             '12/Aug/1995': 194,
             '13/Aug/1995': 216,
             '14/Aug/1995': 284,
             '15/Aug/1995': 326,
             '16/Aug/1995': 257,
             '17/Aug/1995': 269,
             '18/Aug/1995': 255,
             '19/Aug/1995': 207,
             '20/Aug/1995': 312,
             '21/Aug/1995': 305,
             '22/Aug/1995': 287,
             '23/Aug/1995': 345,
             '24/Aug/1995': 420,
             '25/Aug/1995': 412,
             '26/Aug/1995': 366,
             '27/Aug/1995': 370,
             '28/Aug/1995': 409,
             '29/Aug/1995': 420,
             '30/Aug/1995': 567,
             '31/Aug/1995'

#### 4.1: Quantidade de erros 404 por dia em julho
#### Recuperar os dias do campo timestamp e contar os erros 404

In [42]:
erro_diaJU = rdd_julho_clean.filter(lambda line: verifica(line)) \
                            .map(lambda line: line[1::2]) \
                            .map(lambda line: ajusta_time(line)) \
                            .map(lambda line: line[0]).countByValue()

In [46]:
erro_diaJU

defaultdict(int,
            {'01/Jul/1995': 315,
             '02/Jul/1995': 289,
             '03/Jul/1995': 474,
             '04/Jul/1995': 357,
             '05/Jul/1995': 494,
             '06/Jul/1995': 635,
             '07/Jul/1995': 565,
             '08/Jul/1995': 299,
             '09/Jul/1995': 347,
             '10/Jul/1995': 394,
             '11/Jul/1995': 469,
             '12/Jul/1995': 461,
             '13/Jul/1995': 524,
             '14/Jul/1995': 410,
             '15/Jul/1995': 253,
             '16/Jul/1995': 256,
             '17/Jul/1995': 405,
             '18/Jul/1995': 463,
             '19/Jul/1995': 639,
             '20/Jul/1995': 427,
             '21/Jul/1995': 334,
             '22/Jul/1995': 182,
             '23/Jul/1995': 233,
             '24/Jul/1995': 327,
             '25/Jul/1995': 459,
             '26/Jul/1995': 331,
             '27/Jul/1995': 333,
             '28/Jul/1995': 94})

#### 5: O total de bytes retornados
#### (Somar toda a coluna bytes retornados em agosto)

In [ ]:
# Cria-se uma lista vazia
# Essa lista recebe os valores correspondentes aos índice 4
# Os valores somente são inseridos caso a lista tenha 5 posições
# Os valores são somados e impressos na tela

In [18]:
lista_agosto = []
for x in rdd_agosto_clean.take(rdd_agosto_clean.count()):
    if len(x) == 5:
        lista_agosto.append(int(x[4]))
    
total_bytes_agosto = sum(lista_agosto)
print("Total de bytes retornados em agosto:", total_bytes_agosto)

Total de bytes retornados em agosto: 26711359975


#### 5.1: O total de bytes retornados
#### (Somar toda a coluna bytes retornados em agosto em julho)

In [19]:
lista_julho = []
for x in rdd_julho_clean.take(rdd_julho_clean.count()):
    if len(x) == 5:
        lista_julho.append(int(x[4]))
    
total_bytes_julho = sum(lista_julho)
print("Total de bytes retornados em julho:", total_bytes_julho)

Total de bytes retornados em julho: 38548546838
